In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 공통 코드

In [ ]:
import os
import sys
!pip install -r '/content/drive/MyDrive/gas/#requirements_GasDemand/requirements_GasDemand.txt' # drive path 수정
!pip uninstall -y typing # this should avoid  AttributeError: type object 'Callable' has no attribute '_abc_registry'
# !pip install  "git+https://github.com/dreamquark-ai/tabnet.git@develop#egg=pytorch_tabnet" --upgrade

In [ ]:
import warnings; warnings.filterwarnings(action='ignore')
from typing import Dict, Tuple, Union, List
from tqdm import tqdm

import numpy as np
import pandas as pd
from datetime import datetime
import torch

import seaborn as sns
import matplotlib as mlt
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_palette('deep')
sns.set_color_codes()
sns.set_style('darkgrid')

# Models
## Timeseries Models
from fbprophet import Prophet

## Tree Models
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
import xgboost as XGBRegressor
import xgboost as xgb

## Meta Model
# from pytorch_tabnet.tab_model import TabNetRegressor

# Preprocessing
from sklearn.model_selection import StratifiedKFold

# Metrics
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE

# etc
import holidays
from termcolor import colored

# Customs
sys.path.append('/content/drive/MyDrive/gas/#requirements_GasDemand')
from custom_functions import CreateMatch_test_features
from custom_functions import HolidayMatch_train_test

In [ ]:
#path = '/content/drive/MyDrive/my_data/Gas_Demand/' # data path 수정
path="/content/drive/MyDrive/gas/"
train = pd.read_csv(path+"new_total_stl.csv")
f_train = pd.read_csv(path+"train.csv",encoding='cp949')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

test = CreateMatch_test_features(test)
train, test = HolidayMatch_train_test(train, test)

In [ ]:
train.drop(['시간블록'], axis = 1, inplace= True)
test.drop(['시간블록'], axis = 1, inplace= True)
train['시간'] += 1
train.dropna(axis = 0, inplace=True)

In [ ]:
train.head(3)

,연월일,시간,구분,month,year,weekday,holiday,공급량
0,2013-01-01,1,A,1,2013,1,True,2497.129
1,2013-01-01,2,A,1,2013,1,True,2363.265
2,2013-01-01,3,A,1,2013,1,True,2258.505


In [ ]:
test.head(3)

,연월일,시간,구분,month,year,weekday,holiday
0,2019-01-01,1,A,1,2019,1,True
1,2019-01-01,2,A,1,2019,1,True
2,2019-01-01,3,A,1,2019,1,True


In [ ]:
for i in set(train['구분']):
    globals()['train_{}'.format(i)] = train[train['구분'] == i].drop(['구분'], axis = 1).reset_index(drop = True)
    globals()['test_{}'.format(i)] = test[test['구분'] == i].drop(['구분'], axis = 1)

# 개인별 분석 시작

* 위에까지는 성연이가 짜놓은 코드를 입출력 동일하게 해놓은 것입니다.

* 라이브러리 임포트 부분은 #과같은 형식을 갖춰서 마음껏 임포트합시다.

* 설치해야할 패키지가 있다면 requirementes_GasDemand.txt 파일에 install 다음 부분만 적어놓으시면 됩니다. 정 안되시면 그냥 !pip 해둡시다.

* 아래부터는 개인적인 코드 사용가능합니다.

* 같이 일하는 사람이 많으니 함수명이나 변수명등을 지을때 최대한 서로 보기 쉽게 짜놓을 수 있으면 좋겠습니다.

* 각자의 Custom 라이브러리를 만들려는 이유는 서로 코드 보기 편하자고 하는 것이 목적입니다. 그러니 간단한 함수가 필요하다거나 라이브러리 만드는 방법이 너무 힘들다 싶으면 그냥 안하셔도 됩니다. 대신, 깔끔했으면 좋겠어요 

# 기온 분석

In [ ]:
pd.read_csv(path+'/new_weather/weather2013.csv',encoding='cp949')

,format: day,hour,value location:60_127 Start : 20130101
0,1,0.0,-6.9
1,1,100.0,-5.4
2,1,200.0,-4.1
3,1,300.0,-4.1
4,1,400.0,-1.8
...,...,...,...
8766,31,1900.0,0.9
8767,31,2000.0,0.6
8768,31,2100.0,0.7
8769,31,2200.0,0.7


In [ ]:
pd.read_csv(path+'/new_weather/weather2014.csv',encoding='cp949')

,format: day,hour,value location:60_127 Start : 20140101
0,1,0.0,1.8
1,1,100.0,3.6
2,1,200.0,5.4
3,1,300.0,7.0
4,1,400.0,6.7
...,...,...,...
8766,31,1900.0,-8.7
8767,31,2000.0,-9.0
8768,31,2100.0,-9.2
8769,31,2200.0,-9.4


In [ ]:
365 * 24

8760

In [ ]:
f_train[f_train['구분'] == 'A'].iloc[:8760,:]

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105
...,...,...,...,...
8755,2013-12-31,20,A,2298.889
8756,2013-12-31,21,A,2185.665
8757,2013-12-31,22,A,2097.929
8758,2013-12-31,23,A,1999.705


In [ ]:
train_A[train_A['year'] == 2013]

,연월일,시간,month,year,weekday,holiday,공급량,group_day,시간당비율
0,2013-01-01,1,1,2013,1,True,2497.129,57606.415,0.043348
1,2013-01-01,2,1,2013,1,True,2363.265,57606.415,0.041024
2,2013-01-01,3,1,2013,1,True,2258.505,57606.415,0.039206
3,2013-01-01,4,1,2013,1,True,2243.969,57606.415,0.038953
4,2013-01-01,5,1,2013,1,True,2344.105,57606.415,0.040692
...,...,...,...,...,...,...,...,...,...
8739,2013-12-31,20,12,2013,1,False,2298.889,49663.144,0.046290
8740,2013-12-31,21,12,2013,1,False,2185.665,49663.144,0.044010
8741,2013-12-31,22,12,2013,1,False,2097.929,49663.144,0.042243
8742,2013-12-31,23,12,2013,1,False,1999.705,49663.144,0.040265


In [ ]:
#데이터 불러오기
temp_train=pd.DataFrame()
temp_test=pd.DataFrame()
for year in [2012,2013,2014,2015,2016,2017,2018]:
  temp_train=temp_train.append(pd.read_csv(path+'/weather/'+f'weather{year}.csv',encoding='cp949'))
temp_train.reset_index(drop=True,inplace=True)
for year in [2019]:
  temp_test=temp_test.append(pd.read_csv(path+'/weather/'+f'weather{year}.csv',encoding='cp949'))
temp_test.reset_index(drop=True,inplace=True)

#데이터 최고기온, 최저기온으로 전처리
temp_train['연월일'] = temp_train['일시'].str.split(' ').str[0]
temp_train['시간'] = temp_train['일시'].str.split(' ').str[1]
temp_test['연월일'] = temp_test['일시'].str.split(' ').str[0]
temp_test['시간'] = temp_test['일시'].str.split(' ').str[1]
temp_train.drop(['지점','지점명','일시','기온 QC플래그'],axis=1,inplace=True)
temp_test.drop(['지점','지점명','일시'],axis=1,inplace=True)

temp_train=pd.DataFrame({'연월일':temp_train.연월일.unique(),'최고기온':temp_train.groupby('연월일')['기온(°C)'].max().values,'최저기온':temp_train.groupby('연월일')['기온(°C)'].min().values})
temp_test=pd.DataFrame({'연월일':temp_test.연월일.unique(),'최고기온':temp_test.groupby('연월일')['기온(°C)'].max().values,'최저기온':temp_test.groupby('연월일')['기온(°C)'].min().values})

temp_train.연월일=pd.to_datetime(temp_train.연월일)
temp_test.연월일=pd.to_datetime(temp_test.연월일)

# 시간대 별 비율 찾기

In [ ]:
def preprocess(train):
    train['group_day']=0
    data=pd.DataFrame({'연월일':train.연월일.unique(),'group_day':train.groupby(['연월일']).공급량.sum().values})
    for i in tqdm(range(data.shape[0])):
        train.loc[train.연월일==data.연월일[i],'group_day']=data.group_day[i]
    train['시간당비율'] = train['공급량'] / train['group_day']
    return train

In [ ]:
train_A = preprocess(train_A)
train_A_tlabel = train_A['시간당비율']
train_A_t = train_A.drop(['연월일','공급량','year','group_day','시간당비율'], axis = 1)
test_A_t = test_A.drop(['연월일','year'], axis = 1)
rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(train_A_t,train_A_tlabel)
test_A['시간당비율'] = rf.predict(test_A_t)

100%|██████████| 2191/2191 [00:02<00:00, 958.98it/s]


In [ ]:
train_B = preprocess(train_B)
train_B_tlabel = train_B['시간당비율']
train_B_t = train_B.drop(['연월일','공급량','year','group_day','시간당비율'], axis = 1)
test_B_t = test_B.drop(['연월일','year'], axis = 1)
rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(train_B_t,train_B_tlabel)
test_B['시간당비율'] = rf.predict(test_B_t)

100%|██████████| 2191/2191 [00:02<00:00, 967.32it/s]


In [ ]:
train_C = preprocess(train_C)
train_C_tlabel = train_C['시간당비율']
train_C_t = train_C.drop(['연월일','공급량','year','group_day','시간당비율'], axis = 1)
test_C_t = test_C.drop(['연월일','year'], axis = 1)
rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(train_C_t,train_C_tlabel)
test_C['시간당비율'] = rf.predict(test_C_t)

100%|██████████| 2191/2191 [00:02<00:00, 989.48it/s]


In [ ]:
train_D = preprocess(train_D)
train_D_tlabel = train_D['시간당비율']
train_D_t = train_D.drop(['연월일','공급량','year','group_day','시간당비율'], axis = 1)
test_D_t = test_D.drop(['연월일','year'], axis = 1)
rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(train_D_t,train_D_tlabel)
test_D['시간당비율'] = rf.predict(test_D_t)

100%|██████████| 2191/2191 [00:02<00:00, 992.15it/s]


In [ ]:
train_E = preprocess(train_E)
train_E_tlabel = train_E['시간당비율']
train_E_t = train_E.drop(['연월일','공급량','year','group_day','시간당비율'], axis = 1)
test_E_t = test_E.drop(['연월일','year'], axis = 1)
rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(train_E_t,train_E_tlabel)
test_E['시간당비율'] = rf.predict(test_E_t)

100%|██████████| 2191/2191 [00:02<00:00, 992.98it/s]


In [ ]:
train_G = preprocess(train_G)
train_G_tlabel = train_G['시간당비율']
train_G_t = train_G.drop(['연월일','공급량','year','group_day','시간당비율'], axis = 1)
test_G_t = test_G.drop(['연월일','year'], axis = 1)
rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(train_G_t,train_G_tlabel)
test_G['시간당비율'] = rf.predict(test_G_t)

100%|██████████| 2191/2191 [00:02<00:00, 984.02it/s]


In [ ]:
train_H = preprocess(train_H)
train_H_tlabel = train_H['시간당비율']
train_H_t = train_H.drop(['연월일','공급량','year','group_day','시간당비율'], axis = 1)
test_H_t = test_H.drop(['연월일','year'], axis = 1)
rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(train_H_t,train_H_tlabel)
test_H['시간당비율'] = rf.predict(test_H_t)

100%|██████████| 2191/2191 [00:02<00:00, 996.53it/s] 


# 일자별 공급량 찾기

In [ ]:
day_train_A=train_A[['연월일','month','year','weekday','holiday','group_day']].drop_duplicates()
day_test_A=test_A[['연월일','month','year','weekday','holiday']].drop_duplicates()
day_train_A_label = np.log1p(day_train_A['group_day'])
day_train_A.drop(['연월일','year','group_day'], axis = 1, inplace= True)

rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(day_train_A,day_train_A_label)
day_test_A['group_day'] = np.expm1(rf.predict(day_test_A.drop(['연월일','year'], axis = 1)))
test_A = pd.merge(test_A, day_test_A, how='left')

In [ ]:
day_train_B=train_B[['연월일','month','year','weekday','holiday','group_day']].drop_duplicates()
day_test_B=test_B[['연월일','month','year','weekday','holiday']].drop_duplicates()
day_train_B_label = np.log1p(day_train_B['group_day'])
day_train_B.drop(['연월일','year','group_day'], axis = 1, inplace= True)

rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(day_train_B,day_train_B_label)
day_test_B['group_day'] = np.expm1(rf.predict(day_test_B.drop(['연월일','year'], axis = 1)))
test_B = pd.merge(test_B, day_test_B, how='left')

In [ ]:
day_train_C=train_C[['연월일','month','year','weekday','holiday','group_day']].drop_duplicates()
day_test_C=test_C[['연월일','month','year','weekday','holiday']].drop_duplicates()
day_train_C_label = np.log1p(day_train_C['group_day'])
day_train_C.drop(['연월일','year','group_day'], axis = 1, inplace= True)

rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(day_train_C,day_train_C_label)
day_test_C['group_day'] = np.expm1(rf.predict(day_test_C.drop(['연월일','year'], axis = 1)))
test_C = pd.merge(test_C, day_test_C, how='left')

In [ ]:
day_train_D=train_D[['연월일','month','year','weekday','holiday','group_day']].drop_duplicates()
day_test_D=test_D[['연월일','month','year','weekday','holiday']].drop_duplicates()
day_train_D_label = np.log1p(day_train_D['group_day'])
day_train_D.drop(['연월일','year','group_day'], axis = 1, inplace= True)

rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(day_train_D,day_train_D_label)
day_test_D['group_day'] = np.expm1(rf.predict(day_test_D.drop(['연월일','year'], axis = 1)))
test_D = pd.merge(test_D, day_test_D, how='left')

In [ ]:
day_train_E=train_E[['연월일','month','year','weekday','holiday','group_day']].drop_duplicates()
day_test_E=test_E[['연월일','month','year','weekday','holiday']].drop_duplicates()
day_train_E_label = np.log1p(day_train_E['group_day'])
day_train_E.drop(['연월일','year','group_day'], axis = 1, inplace= True)

rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(day_train_E,day_train_E_label)
day_test_E['group_day'] = np.expm1(rf.predict(day_test_E.drop(['연월일','year'], axis = 1)))
test_E = pd.merge(test_E, day_test_E, how='left')

In [ ]:
day_train_G=train_G[['연월일','month','year','weekday','holiday','group_day']].drop_duplicates()
day_test_G=test_G[['연월일','month','year','weekday','holiday']].drop_duplicates()
day_train_G_label = np.log1p(day_train_G['group_day'])
day_train_G.drop(['연월일','year','group_day'], axis = 1, inplace= True)

rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(day_train_G,day_train_G_label)
day_test_G['group_day'] = np.expm1(rf.predict(day_test_G.drop(['연월일','year'], axis = 1)))
test_G = pd.merge(test_G, day_test_G, how='left')

In [ ]:
day_train_H=train_H[['연월일','month','year','weekday','holiday','group_day']].drop_duplicates()
day_test_H=test_H[['연월일','month','year','weekday','holiday']].drop_duplicates()
day_train_H_label = np.log1p(day_train_H['group_day'])
day_train_H.drop(['연월일','year','group_day'], axis = 1, inplace= True)

rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(day_train_H,day_train_H_label)
day_test_H['group_day'] = np.expm1(rf.predict(day_test_H.drop(['연월일','year'], axis = 1)))
test_H = pd.merge(test_H, day_test_H, how='left')

# 최종 계산

In [ ]:
new_test = pd.concat([test_A, test_B, test_C, test_D, test_E, test_G, test_H])

submission['공급량'] = (new_test['시간당비율'] * new_test['group_day']).reset_index(drop = True)
submission.to_csv('gas_KSY_20.csv',encoding='UTF-8',index=False)

In [ ]:
submission['공급량'].mean()

1547.1716566119158